In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.express as px
import nbformat

## Fetch some stock data
Let's go with AAPL.

In [2]:
AAPL = yf.download("AAPL",period="2y",progress=False)
AAPL

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-14,150.350006,151.070007,146.910004,148.119995,146.425964,109296300
2021-09-15,148.559998,149.440002,146.369995,149.029999,147.325577,83281300
2021-09-16,148.440002,148.970001,147.220001,148.789993,147.088303,68034100
2021-09-17,148.820007,148.820007,145.759995,146.059998,144.389526,129868800
2021-09-20,143.800003,144.839996,141.270004,142.940002,141.305222,123478900
...,...,...,...,...,...,...
2023-09-07,175.179993,178.210007,173.539993,177.559998,177.559998,112488800
2023-09-08,178.350006,180.240005,177.789993,178.179993,178.179993,65551300
2023-09-11,180.070007,180.300003,177.339996,179.360001,179.360001,58953100


## Visualize the stock price

In [3]:
fig = px.line(AAPL, y="Adj Close", title='AAPL Stock Price', labels = {'Adj Close':'AAPL Close Price(in USD)'})

In [4]:
fig.show()

## Moving Average 1 (Short window)

Here I am choosing Exponential moving average instead of Simple Moving Average, feel free to change it to SMA instead of EMA, you can do so in the following way.
```python
ema1['Adj Close'] = AAPL['Adj Close'].ewm(span = window1).mean()

```

In [5]:
window1 = 30
sma1 = pd.DataFrame()
sma1['Adj Close'] = AAPL['Adj Close'].rolling(window = window1).mean()
sma1

,Adj Close
Date,
2021-09-14,NaN
2021-09-15,NaN
2021-09-16,NaN
2021-09-17,NaN
2021-09-20,NaN
...,...
2023-09-07,182.909325
2023-09-08,182.416677
2023-09-11,181.876480


## Moving Average 2 (Long Window)

In [6]:
window2 = 100
sma2 = pd.DataFrame()
sma2['Adj Close'] = AAPL['Adj Close'].rolling(window = window2).mean()
sma2

,Adj Close
Date,
2021-09-14,NaN
2021-09-15,NaN
2021-09-16,NaN
2021-09-17,NaN
2021-09-20,NaN
...,...
2023-09-07,180.582149
2023-09-08,180.716156
2023-09-11,180.849598


In [7]:
fig.add_scatter(x=sma1.index,y=sma1['Adj Close'], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=sma2.index,y=sma2['Adj Close'], mode='lines',name='SMA'+str(window2))
fig.show()

## Combine everything

In [8]:
data = pd.DataFrame()
data['AAPL'] = AAPL['Adj Close']
data['SMA'+str(window1)] = sma1['Adj Close']
data['SMA'+str(window2)] = sma2['Adj Close']
data

,AAPL,SMA30,SMA100
Date,,,
2021-09-14,146.425964,NaN,NaN
2021-09-15,147.325577,NaN,NaN
2021-09-16,147.088303,NaN,NaN
2021-09-17,144.389526,NaN,NaN
2021-09-20,141.305222,NaN,NaN
...,...,...,...
2023-09-07,177.559998,182.909325,180.582149
2023-09-08,178.179993,182.416677,180.716156
2023-09-11,179.360001,181.876480,180.849598


## Strategy to generate buy/sell signal

In [9]:
def dualMACrossover(data):
    sigPriceBuy = []
    sigPriceSell = []
    flag = -1 # Flag denoting when the 2 moving averages crossed each other
    for i in range(len(data)):
        if data['SMA'+str(window1)][i] > data['SMA'+str(window2)][i]:
            if flag != 1:
                sigPriceBuy.append(data['AAPL'][i])
                sigPriceSell.append(np.nan)
                flag = 1
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        elif data['SMA'+str(window1)][i] < data['SMA'+str(window2)][i]:
            if flag!=0:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(data['AAPL'][i])
                flag=0
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        else:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
    return (sigPriceBuy,sigPriceSell)

In [10]:
buy_sell = dualMACrossover(data)
data['BuySignalPrice'] = buy_sell[0]
data['SellSignalPrice'] = buy_sell[1]

## Visualize the data and the strategy

In [11]:
import plotly.graph_objects as go

fig = px.line(data, y="AAPL", title='Strategy Visualization', labels = {'index':'Date'})
fig.add_scatter(x=data.index,y=data['SMA'+str(window1)], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=data.index,y=data['SMA'+str(window2)], mode='lines',name='SMA'+str(window2))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.BuySignalPrice, marker_symbol='triangle-up',
                           marker_line_color="#000000", marker_color="#000000",
                           marker_line_width=2, marker_size=15, name='Buy'))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.SellSignalPrice, marker_symbol='triangle-down',
                           marker_line_color="#E74C3C", marker_color="#E74C3C",
                           marker_line_width=2, marker_size=15, name='Sell'))
fig.show()

## Backtest the strategy

In [12]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import backtesting
backtesting.set_bokeh_output(notebook=False)

c:\Users\Dung03\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [13]:
class DualMACrossover(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, window1)
        self.ma2 = self.I(SMA, price, window2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(AAPL, DualMACrossover,
              exclusive_orders=True)
stats = bt.run()

In [14]:
stats

Start                     2021-09-14 00:00:00
End                       2023-09-13 00:00:00
Duration                    729 days 00:00:00
Exposure Time [%]                    74.15507
Equity Final [$]                    8193.5383
Equity Peak [$]                  12040.449814
Return [%]                         -18.064617
Buy & Hold Return [%]               17.614105
Return (Ann.) [%]                    -9.49976
Volatility (Ann.) [%]               23.587179
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -44.470189
Avg. Drawdown [%]                   -9.274931
Max. Drawdown Duration      454 days 00:00:00
Avg. Drawdown Duration       68 days 00:00:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                      15.858213
Worst Trade [%]                    -18.884094
Avg. Trade [%]                    

# Exercise


* What is the return and annual return of the above algorithm?


Return: -18.064617 <br>
Annual return: -9.49976

* What is the annual volatility of the stock?


Annual volatility: 23.587179

* Backtest the same strategy for TESLA stock and calculate the return, annual return, and annual volatility


In [18]:
TSLA = yf.download('TSLA', period='2y', progress=False)
TSLA

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-14,247.523331,251.490005,245.466660,248.163330,248.163330,55574700
2021-09-15,248.333328,252.286667,246.119995,251.943329,251.943329,46073100
2021-09-16,250.943329,252.970001,249.203339,252.330002,252.330002,41770200
2021-09-17,252.383331,253.679993,250.000000,253.163330,253.163330,84612600
2021-09-20,244.853333,247.333328,239.539993,243.389999,243.389999,74273100
...,...,...,...,...,...,...
2023-09-07,245.070007,252.809998,243.270004,251.490005,251.490005,115312900
2023-09-08,251.220001,256.519989,246.669998,248.500000,248.500000,118367700
2023-09-11,264.269989,274.850006,260.609985,273.579987,273.579987,174667900


In [20]:
fig = px.line(TSLA, y="Adj Close", title='TESLA Stock Price', labels = {'Adj Close':'TESLA Close Price(in USD)'})
fig.show()

In [23]:
window1 = 30
sma1 = pd.DataFrame()
sma1['Adj Close'] = TSLA['Adj Close'].rolling(window = window1).mean()
sma1

,Adj Close
Date,
2021-09-14,NaN
2021-09-15,NaN
2021-09-16,NaN
2021-09-17,NaN
2021-09-20,NaN
...,...
2023-09-07,245.604999
2023-09-08,245.364666
2023-09-11,245.602665


In [24]:
window2 = 100
sma2 = pd.DataFrame()
sma2['Adj Close'] = TSLA['Adj Close'].rolling(window = window2).mean()
sma2

,Adj Close
Date,
2021-09-14,NaN
2021-09-15,NaN
2021-09-16,NaN
2021-09-17,NaN
2021-09-20,NaN
...,...
2023-09-07,227.656399
2023-09-08,228.270999
2023-09-11,229.163699


In [25]:
fig.add_scatter(x=sma1.index,y=sma1['Adj Close'], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=sma2.index,y=sma2['Adj Close'], mode='lines',name='SMA'+str(window2))
fig.show()

In [26]:
data = pd.DataFrame()
data['TSLA'] = TSLA['Adj Close']
data['SMA'+str(window1)] = sma1['Adj Close']
data['SMA'+str(window2)] = sma2['Adj Close']
data

,TSLA,SMA30,SMA100
Date,,,
2021-09-14,248.163330,NaN,NaN
2021-09-15,251.943329,NaN,NaN
2021-09-16,252.330002,NaN,NaN
2021-09-17,253.163330,NaN,NaN
2021-09-20,243.389999,NaN,NaN
...,...,...,...
2023-09-07,251.490005,245.604999,227.656399
2023-09-08,248.500000,245.364666,228.270999
2023-09-11,273.579987,245.602665,229.163699


In [29]:
def dualMACrossover(data):
    sigPriceBuy = []
    sigPriceSell = []
    flag = -1 # Flag denoting when the 2 moving averages crossed each other
    for i in range(len(data)):
        if data['SMA'+str(window1)][i] > data['SMA'+str(window2)][i]:
            if flag != 1:
                sigPriceBuy.append(data['TSLA'][i])
                sigPriceSell.append(np.nan)
                flag = 1
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        elif data['SMA'+str(window1)][i] < data['SMA'+str(window2)][i]:
            if flag!=0:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(data['TSLA'][i])
                flag=0
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        else:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
    return (sigPriceBuy,sigPriceSell)

In [30]:
buy_sell = dualMACrossover(data)
data['BuySignalPrice'] = buy_sell[0]
data['SellSignalPrice'] = buy_sell[1]

In [31]:
import plotly.graph_objects as go

fig = px.line(data, y="TSLA", title='Strategy Visualization', labels = {'index':'Date'})
fig.add_scatter(x=data.index,y=data['SMA'+str(window1)], mode='lines',name='SMA'+str(window1))
fig.add_scatter(x=data.index,y=data['SMA'+str(window2)], mode='lines',name='SMA'+str(window2))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.BuySignalPrice, marker_symbol='triangle-up',
                           marker_line_color="#000000", marker_color="#000000",
                           marker_line_width=2, marker_size=15, name='Buy'))

fig.add_trace(go.Scatter(mode="markers", x=data.index, y=data.SellSignalPrice, marker_symbol='triangle-down',
                           marker_line_color="#E74C3C", marker_color="#E74C3C",
                           marker_line_width=2, marker_size=15, name='Sell'))
fig.show()

In [32]:
bt = Backtest(TSLA, DualMACrossover,
              exclusive_orders=True)
stats = bt.run()

In [33]:
stats

Start                     2021-09-14 00:00:00
End                       2023-09-13 00:00:00
Duration                    729 days 00:00:00
Exposure Time [%]                   78.926441
Equity Final [$]                  2272.125809
Equity Peak [$]                  11105.340332
Return [%]                         -77.278742
Buy & Hold Return [%]                9.323157
Return (Ann.) [%]                  -52.403342
Volatility (Ann.) [%]               25.988423
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -83.472044
Avg. Drawdown [%]                  -45.239154
Max. Drawdown Duration      567 days 00:00:00
Avg. Drawdown Duration      287 days 00:00:00
# Trades                                    8
Win Rate [%]                             25.0
Best Trade [%]                       8.431364
Worst Trade [%]                    -37.836199
Avg. Trade [%]                    

Return: -77.278742 <br>
Annual return: -52.403342 <br>
Annual volatility: 25.988423

* Adjust the lengths of short window and long window (long window > short window) and see if you can increase the annual return